In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql.functions import col, explode, lit, array, least, greatest


pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

spark = SparkSession.builder.getOrCreate()

In [0]:
# Read profiles data called "profiles"

In [0]:
# Collect data in smaller partitions
url_list = []
for row in profiles.select("url").toLocalIterator():
    url_list.append(row["url"])

In [0]:
!pip install playwright
!playwright install
!pip install nest_asyncio
!pip install selenium



  Obtaining dependency information for playwright from https://files.pythonhosted.org/packages/9a/c1/ea765e72a746dc7ec2ce155ffea29d454e7171db78f3c09185e888387246/playwright-1.49.1-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for greenlet==3.1.1 from https://files.pythonhosted.org/packages/f7/4b/1c9695aa24f808e156c8f4813f685d975ca73c000c2a5056c514c64980f6/greenlet-3.1.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for pyee==12.0.0 from https://files.pythonhosted.org/packages/1d/0d/95993c08c721ec68892547f2117e8f9dfbcef2ca71e098533541b4a54d5f/pyee-12.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.2 MB ? eta -:--:--
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/44.2 MB 46.0 MB/s eta 0:00:01
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/44.2 MB 126.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/44.2 MB 224.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━

In [0]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

nest_asyncio.apply()  # Patch the event loop for Databricks

async def scrape_user_profile(url_list):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        scraped_data = {}
        skills_list = []

        for url in url_list:
            skills_url = f"{url}/details/skills/"
            try:
                print(f"Navigating to: {skills_url}")
                await page.goto(skills_url)
                await page.wait_for_selector("a[data-field='skill_page_skill_topic']", timeout=10000)

                # Get HTML content
                html = await page.content()
                scraped_data["skills"] = html

                # Parse the HTML
                soup_skill = BeautifulSoup(html, "html.parser")
                skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})

                # Extract skills
                skills = [
                    link.find("span", {"aria-hidden": "true"}).get_text(strip=True)
                    for link in skill_links if link.find("span", {"aria-hidden": "true"})
                ]
                unique_skills = sorted(set(skills))
                print(f"Unique skills for {url}: {unique_skills}")
                skills_list.append(unique_skills)

            except Exception as e:
                print(f"Error scraping {skills_url}: {e}")

        await browser.close()
        return skills_list

# Use `await` to run the async function in Databricks
skills = await scrape_user_profile(url_list)
print("Skills:", skills)

Navigating to: https://www.linkedin.com/in/glenn-wagner-366824138/details/skills/


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

עובד בלי להגיע לעמוד עצמו - הודעת שגיאה בדף

In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from PIL import Image


# Proxy Configuration
PROXY = ###
USERNAME = ###
PASSWORD = ###

# Proxy authentication embedded in the URL
PROXY_URL = f"http://{USERNAME}:{PASSWORD}@{PROXY}"

# Chrome Options
chrome_options = Options()
chrome_options.add_argument("--proxy-server=%s" % PROXY_URL)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")  # Optional: run in headless mode

print("Connecting to the remote Selenium WebDriver...")
driver = webdriver.Chrome(options=chrome_options)

skills_list = []

try:
    for url in url_list:
        
        print(f"Scraping profile: {url}")
        
        # Navigate to the LinkedIn profile
        driver.get(url)
        time.sleep(5)  # Allow page to load; adjust if necessary

        # Navigate to the skills section
        skills_url = f"{url}/details/skills/"
        driver.get(skills_url)
        print(f"Navigated to skills page: {skills_url}")

        try:
            print('break 0')

            # Wait for the skills to load
            WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div"))
)
            print('break 1')

            driver.save_screenshot('skills_page.png')

            # Show the screenshot using Pillow
            img = Image.open('skills_page.png')
            img.show()

            # Parse the page source using BeautifulSoup
            html = driver.page_source
            soup_skill = BeautifulSoup(html, "html.parser")
            skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})
            print('break 2')

            # Extract skills
            skills = [
                link.find("span", {"aria-hidden": "true"}).get_text(strip=True)
                for link in skill_links if link.find("span", {"aria-hidden": "true"})
            ]

            print('break 3')

            if skills:
                unique_skills = sorted(set(skills))
                print(f"Skills found: {unique_skills}")
                skills_list.append({url: unique_skills})
            else:
                print(f"No skills found for {url}")
                skills_list.append('')
            
            print('break 4')
  
        except Exception as e:
            print(f"Error scraping skills for {url}: {e}")
   
except Exception as e:
    print(f"Error: {e}")
    
finally:
    # Close the browser
    driver.quit()

# Output the results
print("All Skills:", skills_list)


Connecting to the remote Selenium WebDriver...
Scraping profile: https://www.linkedin.com/in/glenn-wagner-366824138
Navigated to skills page: https://www.linkedin.com/in/glenn-wagner-366824138/details/skills/
break 0
break 1
break 2
break 3
No skills found for https://www.linkedin.com/in/glenn-wagner-366824138
break 4
Scraping profile: https://www.linkedin.com/in/stephen-hockenbarger-494128210


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpoggm772t.PNG'


Navigated to skills page: https://www.linkedin.com/in/stephen-hockenbarger-494128210/details/skills/
break 0
break 1
break 2
break 3
No skills found for https://www.linkedin.com/in/stephen-hockenbarger-494128210
break 4
Scraping profile: https://www.linkedin.com/in/navin-kulkarni-515a8028


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmp7384k3op.PNG'


Navigated to skills page: https://www.linkedin.com/in/navin-kulkarni-515a8028/details/skills/
break 0
break 1
break 2
break 3
No skills found for https://www.linkedin.com/in/navin-kulkarni-515a8028
break 4
Scraping profile: https://www.linkedin.com/in/megan-dorsey-7ab6a9167


Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmp801mcm2s.PNG'


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

ניסיון נוסף עם פרוקסי

In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from PIL import Image
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options

# Proxy Configuration
PROXY = ###
USERNAME = ###
PASSWORD = ###

# Proxy authentication embedded in the URL
PROXY_AUTH_URL = f"http://{USERNAME}:{PASSWORD}@{PROXY}"

# Configure proxy
proxy = Proxy()
proxy.proxy_type = ProxyType.MANUAL
proxy.http_proxy = f"{USERNAME}:{PASSWORD}@{PROXY}"
proxy.ssl_proxy = f"{USERNAME}:{PASSWORD}@{PROXY}"

# Chrome Options
chrome_options = Options()
chrome_options.add_argument("--proxy-server=http://" + PROXY)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")  # Optional: Run in headless mode

# Add proxy to Chrome Options
chrome_options.add_argument(f"--proxy-auth={PROXY_AUTH_URL}")

# Configure WebDriver
driver = webdriver.Chrome(options=chrome_options)

skills_list = []

try:
    for url in url_list:
        print(f"Scraping profile: {url}")
        
        # Navigate to the LinkedIn profile
        driver.get(url)
        time.sleep(5)

        # Navigate to the skills section
        skills_url = f"{url}/details/skills/"
        driver.get(skills_url)
        print(f"Navigated to skills page: {skills_url}")

        try:
            # Wait for the skills section to load
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.artdeco-card"))
            )

            # Take a screenshot of the skills page
            driver.save_screenshot("skills_page.png")
            print("Screenshot saved as 'skills_page.png'.")

            # Parse the page source using BeautifulSoup
            html = driver.page_source
            soup_skill = BeautifulSoup(html, "html.parser")
            skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})

            # Extract skills
            skills = [
                link.find("span", {"aria-hidden": "true"}).get_text(strip=True)
                for link in skill_links if link.find("span", {"aria-hidden": "true"})
            ]

            if skills:
                unique_skills = sorted(set(skills))
                print(f"Skills found: {unique_skills}")
                skills_list.append({url: unique_skills})
            else:
                print(f"No skills found for {url}")
                skills_list.append('')
        
        except Exception as e:
            print(f"Error scraping skills for {url}: {e}")
    
except Exception as e:
    print(f"Error: {e}")
    
finally:
    driver.quit()

# Output results
print("All Skills:", skills_list)


Scraping profile: https://www.linkedin.com/in/glenn-wagner-366824138
Navigated to skills page: https://www.linkedin.com/in/glenn-wagner-366824138/details/skills/
Error scraping skills for https://www.linkedin.com/in/glenn-wagner-366824138: Message: 

Scraping profile: https://www.linkedin.com/in/stephen-hockenbarger-494128210
Navigated to skills page: https://www.linkedin.com/in/stephen-hockenbarger-494128210/details/skills/
Error scraping skills for https://www.linkedin.com/in/stephen-hockenbarger-494128210: Message: 

Scraping profile: https://www.linkedin.com/in/navin-kulkarni-515a8028
Navigated to skills page: https://www.linkedin.com/in/navin-kulkarni-515a8028/details/skills/
Error scraping skills for https://www.linkedin.com/in/navin-kulkarni-515a8028: Message: 

Scraping profile: https://www.linkedin.com/in/megan-dorsey-7ab6a9167
Navigated to skills page: https://www.linkedin.com/in/megan-dorsey-7ab6a9167/details/skills/
Error scraping skills for https://www.linkedin.com/in/mega

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can